In [1]:
import inspect
from inspect import isgeneratorfunction
from functools import partial
from functools import wraps
from inspect import signature

import unittest
from pathlib import Path
from typing import List
import re
from sections import Rule, RuleSet
import text_reader as tp

# Function Processing Module

Move standard_action, sig_match, and set_method into the module

## Classes as function wrappers
- Class objects are decorators

- Action class
  - Convert *Action* strings into standard functions

- StandardSig class
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.

- FunctionProcessor Class
  - Apply specific Action and StandardSig instances
  - Add attribute indicating if the supplied function is a generator.

### StandardSig Class 
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.
  - Assign annotation to resulting function based on final signature and wrapped function.

A sequence of variable definitions is given representing the desired final signature.

has __call__ method 



Definition
- Final (required) signature 
- Valid signatures
  
> - Without a list of valid signatures it will assume that positional arguments 
>   come in the same order as in the final signature.
> - Additional keyword arguments can be extracted from the 'varkw' argument 
>   (if it exists) in the final signature.
> - Additional keyword arguments can be supplied during instance creation and passed
>   to the called function.

##### Example:

**Required signature:**
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Name: str  |Type: [type, str]|Required: bool  |Positional: bool|Notes: str|
|-----------|-----------------|----------------|----------------|-----------|
|test_object|SourceItem       |True            |True            |Must be present, cannot be in context|
|event      |TriggerEvent     |False           |True            |can be in context?|
|context    |'varkw'          |False           |True            |context  or **context|


**Valid signatures**

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):


|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|


**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

```python
rule_sig = {
    (1, False): lambda func, item, event, context: func(item),
    (1, True): lambda func, item, event, context: func(item, **context),
    (2, False): lambda func, item, event, context: func(item, event),
    (2, True): lambda func, item, event, context: func(item, event, **context),
    (3, False): lambda func, item, event, context: func(item, event, context)
    }
```

### FunctionProcessor Class

- Class objects are decorators / functions that 
  1. Convert *Action* strings into standard functions
  3. Check the function argument signature against expected possible signatures.
  4. Wrap the function with a standard argument signature.
  5. Test whether a supplied function is a generator.
  


- Take text processing functions and convert them into *actions*

- Convert set_method, sig_match, standard_action into a helper class

The class can be initialized with signature and action definitions to create different objects for different function groups. When the object is applied to a function or string, it returns a corresponding function with the expected argument signature.

The class object is a custom decorator function.

[**Emulating callable objects**](https://docs.python.org/3/reference/datamodel.html#:~:text=3.3.6.%20Emulating%20callable%20objects%C2%B6)

`object.__call__(self[, args...])`

Called when the instance is *called* as a function; if this method is defined, 
`x(arg1, arg2, ...)` roughly translates to `type(x).__call__(x, arg1, ...)`.

You can decorate functions with classes - replacing the function with a class instance:

In [1]:
class countCalls(object):
    """ decorator replaces a function with a "countCalls" instance
    which behaves like the original function, but keeps track of calls
    """
    def __init__ (self, functionToTrack):
        self.functionToTrack = functionToTrack
        self.timesCalled = 0
    def __call__ (self, *args, **kwargs):
        self.timesCalled += 1
        return self.functionToTrack(*args, **kwargs)

@countCalls
def doNothing():
    pass

doNothing()
doNothing()
print(doNothing.timesCalled)


2


In [2]:
def decorator_with_args(decorator_to_enhance):
    """
    This function is supposed to be used as a decorator.
    It must decorate an other function, that is intended to be used as a decorator.
    Take a cup of coffee.
    It will allow any decorator to accept an arbitrary number of arguments,
    saving you the headache to remember how to do that every time.
    """
    # We use the same trick we did to pass arguments
    def decorator_maker(*args, **kwargs):
        # We create on the fly a decorator that accepts only a function
        # but keeps the passed arguments from the maker.
        def decorator_wrapper(func):
            # We return the result of the original decorator, which, after all,
            # IS JUST AN ORDINARY FUNCTION (which returns a function).
            # Only pitfall: the decorator must have this specific signature or it won't work:
            return decorator_to_enhance(func, *args, **kwargs)
        return decorator_wrapper
    return decorator_maker


It can be used as follows:

In [3]:
# You create the function you will use as a decorator. And stick a decorator on it :-)
# Don't forget, the signature is "decorator(func, *args, **kwargs)"
@decorator_with_args
def decorated_decorator(func, *args, **kwargs):
    def wrapper(function_arg1, function_arg2):
        print("Decorated with {0} {1}".format(args, kwargs))
        return func(function_arg1, function_arg2)
    return wrapper
# Then you decorate the functions you wish with your brand new decorated decorator.
@decorated_decorator(42, 404, 1024)
def decorated_function(function_arg1, function_arg2):
    print("Hello {0} {1}".format(function_arg1, function_arg2))
decorated_function("Universe and", "everything")
#outputs:
#Decorated with (42, 404, 1024) {}
#Hello Universe and everything
# Whoooot!


Decorated with (42, 404, 1024) {}
Hello Universe and everything


## Allow processing functions to take specific keyword arguments from context

add a wrapper to trap unwanted context items

1. Get parameters and defaults for all arguments of the
   supplied function.
2. Generate a defaults dictionary for all keyword
   arguments.
3. When called, pass non-keyword arguments directly to
   the original function.
4. Update the defaults dictionary with calling
   keywords except the context keyword, if present.
5. If context is among the default keyword arguments
   and is supplied when called, add context to the keyword dictionary; otherwise search the supplied context for any keys in the defaults dictionary and use them to update the defaults dictionary.
10. If a **kwargs is present in the original function,
    pass all remaining context items using **context.



https://stackoverflow.com/questions/76304643/reuse-of-function-signature-type-hints-and-intellisense-args-kwargs


### Standard Actions
*Convert a Method name to a Standard Function.*

Take the name of a standard actions and return the matching function.

**Arguments:**
- action_name (str): The name of a standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises: **
- ValueError If the action_name or method_type supplied are not one of
    the valid action names or method types.

**Returns:**
- (ProcessFunc, RuleFunc): One of the standard action functions.

### sig_match
*Convert the supplied function with a standard signature.*

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):

**Arguments:**
- given_method (MethodOptions): A function to be used as a rule
        method or a process method.
- sig_type (str, optional): The type of argument signature desired.  Can
        be one of: 'Process' or 'Rule'.  Defaults to 'Process'.

**Raises:**
- ValueError If given_method does not have one of the expected
    argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature


### set_method
*Convert the supplied function or action name to a Function with
    the standard signature.*

See the standard_action function for valid action names. See the sig_match
function for valid function argument signatures.
Add a special attribute to the returned function indicating if it is a
generator function.

**Arguments:**
- given_method (RuleMethodOptions): A function, or the name of a
    standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises:**
- ValueError If rule_method is a string and is not one of the
    valid action names, or if rule_method is a function and does not
    have one of the valid argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature

### Method Types

#### rule
- rule_method(item, context) is not allowed; use:
    - rule_method(item, **context)          or
    - rule_method(item, event, context)     instead.

*Required signature:*
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

#### process
- Also applies to Sentinel and Assemble methods.

*Required signature:*
> `process_method(test_object: SourceItem, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item  [, other(s)=defaults(s)])      |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, context)                       |2   |None   |Expected           |
|func(item  [, other(s)] **context)        |2+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.



In [ ]:
#%% Test Text
test_lines = '''
Prescribed dose [cGy]: 5000.0
Prescribed dose [cGy]: not defined

Plan Status: Unapproved
Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal

% for dose(%): 100.0
% for dose (%): not defined
'''


In [ ]:
def parse_prescribed_dose(line, event, **context)->List[List[str]]:# pylint: disable=unused-argument
    '''Split "Prescribed dose [cGy]" into 2 lines:
        Prescribed dose
        Prescribed dose Unit
        '''
    parse_template = [
        ['Prescribed dose', '{dose}'],
        ['Prescribed dose Unit', '{unit}']
        ]
    match_results = event.test_value.groupdict()
    if match_results['dose'] == 'not defined':
        parsed_lines = [
            ['Prescribed dose', ''],
            ['Prescribed dose Unit', '']
            ]
    else:
        parsed_lines = [
            [string_item.format(**match_results) for string_item in line_tmpl]
            for line_tmpl in parse_template
            ]
    for line in parsed_lines:
        yield line


In [ ]:
a = signature(parse_prescribed_dose)
a.parameters

mappingproxy({'line': <Parameter "line">,
              'event': <Parameter "event">,
              'context': <Parameter "**context">})

In [ ]:
list(a.parameters.values())[1].name

'event'

In [ ]:
b = a.bind_partial('l')
b

<BoundArguments (line='l')>

In [ ]:
b.apply_defaults()
b

<BoundArguments (line='l', context={})>

In [ ]:
b.signature

<Signature (line, event, **context) -> List[List[str]]>

In [ ]:
def date_parse(line, event)->List[List[str]]:
    '''If Date,don't split beyond first :'''
    parsed_line = [event.test_value, line.split(':',maxsplit=1)[1]]
    return parsed_line


In [ ]:
def approved_status_parse(line, event)->List[List[str]]:
    '''If Treatment Approved, Split "Plan Status" into 3 lines:
        Plan Status
        Approved on
        Approved by
        '''
    idx1 = line.find(event.test_value)
    idx2 = idx1 + len(event.test_value)
    idx3 = line.find('by')
    idx4 = idx3 + 3
    parsed_lines = [
        ['Plan Status', line[idx1:idx2]],
        ['Approved on', line[idx2:idx3]],
        ['Approved by', line[idx4:]]
        ]
    for line in parsed_lines:
        yield line

In [ ]:
def make_float(item, context):
    item_c = item.replace('Line','')
    try:
        num = float(item_c.strip())
    except:
        return None
    output =  f'Line {num:5.2f}'
    context['num'] = num
    return output

In [ ]:
class TestRuleExceptions(unittest.TestCase):

    @unittest.skip('Currently rule_method(item, context) is allowed')
    def test_no_event_arg(self):
        test_func = lambda item, context: str(item) + repr(context)
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_no_arg(self):
        def test_func():
            return "T"
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_many_arg(self):
        test_func = lambda item1, item2, event, context: 'a'
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_bad_action(self):
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method='Not an Action')

class TestRuleActions(unittest.TestCase):
    def test_original_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Original')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result, test_text)

    def test_event_action(self):
        test_text = 'Test Text'
        sentinel = 'Text'
        test_rule = Rule(sentinel, pass_method='Event')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result.test_value, sentinel)

    def test_none_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='None')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertIsNone(result)

    def test_blank_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Blank')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text)
        self.assertEqual(result, '')

class TestRuleFail(unittest.TestCase):
    def test_fail_method(self):
        test_text = 'Test Line'
        test_rule = Rule('Text', pass_method='Blank',
                            fail_method='Original')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text, {})
        self.assertEqual(result, test_text)
        result2 = test_rule.apply('Test Text', {})
        self.assertEqual(result2, '')
